<a href="https://colab.research.google.com/github/housemLassoued/ML-deployment/blob/main/Detecting_Spam_Emails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df=pd.read_csv('/content/spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df['Message'].iloc[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [6]:
df['Category']=df['Category'].map({'ham':0,'spam':1})

In [7]:
df['Category'].value_counts()

,count
Category,
0,4825
1,747


In [8]:
from sklearn.utils import resample
df_majority=df[df['Category']==0]
df_minority=df[df['Category']==1]
df_minority_upsampled=resample(df_minority,replace=True,n_samples=len(df_majority),random_state=42)
df_upsampled=pd.concat([df_majority,df_minority_upsampled])


In [9]:
df_upsampled['Category'].value_counts()

,count
Category,
0,4825
1,4825


In [10]:
df_upsampled['Message']=df_upsampled['Message'].str.lower()
df_upsampled.head()

,Category,Message
0,0,"go until jurong point, crazy.. available only ..."
1,0,ok lar... joking wif u oni...
3,0,u dun say so early hor... u c already then say...
4,0,"nah i don't think he goes to usf, he lives aro..."
6,0,even my brother is not like to speak with me. ...


In [11]:
df_upsampled['Message']=df_upsampled['Message'].str.replace('[^\w\s]','')
df_upsampled.head()

,Category,Message
0,0,"go until jurong point, crazy.. available only ..."
1,0,ok lar... joking wif u oni...
3,0,u dun say so early hor... u c already then say...
4,0,"nah i don't think he goes to usf, he lives aro..."
6,0,even my brother is not like to speak with me. ...


In [12]:
df_upsampled['Message']=df_upsampled['Message'].str.replace('\d+','')
df_upsampled.head()

,Category,Message
0,0,"go until jurong point, crazy.. available only ..."
1,0,ok lar... joking wif u oni...
3,0,u dun say so early hor... u c already then say...
4,0,"nah i don't think he goes to usf, he lives aro..."
6,0,even my brother is not like to speak with me. ...


In [13]:
import string
punctuations_list = string.punctuation
def remove_punctuations(text):
	temp = str.maketrans('', '', punctuations_list)
	return text.translate(temp)

df_upsampled['Message']= df_upsampled['Message'].apply(lambda x: remove_punctuations(x))
df_upsampled.head()


,Category,Message
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
3,0,u dun say so early hor u c already then say
4,0,nah i dont think he goes to usf he lives aroun...
6,0,even my brother is not like to speak with me t...


In [14]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_text) # Fixed indentation here

df_upsampled['Message']= df_upsampled['Message'].apply(lambda x: remove_stopwords(x))
df_upsampled.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,Category,Message
0,0,go jurong point crazy available bugis n great ...
1,0,ok lar joking wif u oni
3,0,u dun say early hor u c already say
4,0,nah dont think goes usf lives around though
6,0,even brother like speak treat like aids patent


In [15]:
df_upsampled['Message'].iloc[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [19]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    lemmatized_text = [lemmatizer.lemmatize(word) for word in word_tokens]

    return ' '.join(lemmatized_text)

df_upsampled['Message']= df_upsampled['Message'].apply(lambda x: lemmatize_text(x))
df_upsampled.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Category,Message
0,0,go jurong point crazy available bugis n great ...
1,0,ok lar joking wif u oni
3,0,u dun say early hor u c already say
4,0,nah dont think go usf life around though
6,0,even brother like speak treat like aid patent


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_upsampled['Message'])
y = df_upsampled['Category']


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [31]:
model.score(X_test,y_test)

0.9880829015544041

In [32]:
confusionmatrix=pd.crosstab(y_test,model.predict(X_test))
confusionmatrix

col_0,0,1
Category,,
0,975,10
1,13,932


In [33]:
from sklearn.metrics import classification_report
report = classification_report(y_test, model.predict(X_test), target_names=['Classe 0', 'Classe 1'])
print("\nRapport de Classification:")
print(report)


Rapport de Classification:
              precision    recall  f1-score   support

    Classe 0       0.99      0.99      0.99       985
    Classe 1       0.99      0.99      0.99       945

    accuracy                           0.99      1930
   macro avg       0.99      0.99      0.99      1930
weighted avg       0.99      0.99      0.99      1930



In [34]:
from sklearn.ensemble import RandomForestClassifier
model1=RandomForestClassifier()
model1.fit(X_train,y_train)

RandomForestClassifier()

In [35]:
model1.score(X_test,y_test)


0.9994818652849741

In [36]:
from sklearn.metrics import classification_report
report = classification_report(y_test, model1.predict(X_test), target_names=['Classe 0', 'Classe 1'])
print("\nRapport de Classification:")
print(report)


Rapport de Classification:
              precision    recall  f1-score   support

    Classe 0       1.00      1.00      1.00       985
    Classe 1       1.00      1.00      1.00       945

    accuracy                           1.00      1930
   macro avg       1.00      1.00      1.00      1930
weighted avg       1.00      1.00      1.00      1930

